### You are given an n × n binary matrix grid. Your task is to implement and compare two search algorithms to find a path from the top-left cell (0, 0) to the bottom-right cell (n- 1, n - 1).
 A clear path is defined as:
 
 1. All visited cells along the path must have a value of 0.
   2. Moves can be made 8-directionally — i.e., from a cell, you may move to another cell that is horizontally, vertically, or diagonally adjacent.
   3. The length of the path is the total number of visited cells.

# State

In [32]:
class State:
    def __init__(self, grid, i, j,visited):
        self.grid = grid
        self.n = len(grid)
        self.i = i
        self.j = j
        self.visited = visited

    def goalTest(self):
        return self.i == self.n - 1 and self.j == self.n - 1

    def moveGen(self):
        children = []

        if self.grid[self.i][self.j]==1:
            return []
        
        if (0, 0) not in self.visited and self.grid[0][0]==0:
            self.visited = self.visited + [(0, 0)]

        directions = [(1, 1), (1, -1), (-1, 1), (-1, -1), (1, 0), (0, 1),
                      (0, -1), (-1, 0)]
        
        for dr, dc in directions:
            new_i = self.i + dr
            new_j = self.j + dc
            if 0 <= new_i < self.n and 0 <= new_j < self.n and self.grid[new_i][new_j] == 0 and (new_i, new_j) not in self.visited:
                new_visited = self.visited + [(new_i, new_j)]
                children.append(State(self.grid, new_i, new_j, new_visited))
        return children

    def __hash__(self):
        return hash((self.i, self.j))

    def __eq__(self, other):
        return self.i == other.i and self.j == other.j

    def __str__(self):
        return f"({self.i} , {self.j} )"

    #admissible
#     def h(self):
#         return len(self.visited)

    #Euclidean or Manhattan distance to the goal
    def h(self):
        goal_i, goal_j = self.n - 1, self.n - 1
        return abs(goal_i - self.i) + abs(goal_j - self.j)

    def k_step_cost(self, m):
        return 1

# Best First Search

In [33]:
def removeseen(children,open,closed):
    return [node for node in children if node not in open and node not in closed]

def bfs(start):
    open = [start]
    closed = []
    
    while open:
        N = min(open, key = lambda x : x.h())
        open.remove(N)
        
        if N.goalTest():
            return f"Path Length: {len(N.visited)}, Path: {N.visited}"
        else:
            closed.append(N)
            children = N.moveGen()
            new_child = removeseen(children,open,closed)
            open = open + new_child
    
    return "No Path Found"

# A* Search

In [34]:
def improvePropagate(N,parent,g,f,closed):
    for M in N.moveGen():
        if g[N]+N.k_step_cost(M) < g.get(M,float('inf')):
            parent[M] = N
            g[M] = g[N]+N.k_step_cost(M)
            f[M] = g[M]+M.h()
            if M in closed:
                improvePropagate(M,parent,g,f,closed)

def a_star(start):
    parent = {}
    g = {}
    f = {}
    
    open = [start]
    closed = []
    
    parent[start] = None
    g[start] = 0
    f[start] = g[start]+start.h()
    
    while open:
        N = min(open,key = lambda x: f[x])
        open.remove(N)
        
        if N.goalTest():
            return f"Path Length: {len(N.visited)}, Path: {N.visited}"
        else:
            closed.append(N)
            for M in N.moveGen():
                if g[N]+N.k_step_cost(M) < g.get(M,float('inf')): 
                    parent[M] = N
                    g[M] = g[N]+N.k_step_cost(M)
                    f[M] = g[M]+M.h()
                    if M in open:
                        continue
                    elif M in closed:
                        improvePropagate(M,parent,f,g,closed)
                    else:
                        open.append(M)
    return "No path found"

# Test Cases

In [35]:
grid1 = [
    [0, 1],
    [1, 0]
]

grid2 = [
    [0, 0, 0],
    [1, 1, 0],
    [1, 1, 0]
]

grid3 = [
    [1, 0, 0],
    [1, 1, 0],
    [1, 1, 0]
]

state1 = State(grid1,0,0,[])
state2 = State(grid2,0,0,[])
state3 = State(grid3,0,0,[])

# Output

In [36]:
print("grid 1 : ")
print("Best First Search --> ",bfs(state1))
print("A* Search --> ",a_star(state1))

print()

print("grid 2 : ")
print("Best First Search --> ",bfs(state2))
print("A* Search --> ",a_star(state2))

print()

print("grid 3 : ")
print("Best First Search --> ",bfs(state3))
print("A* Search --> ",a_star(state3))

grid 1 : 
Best First Search -->  Path Length: 2, Path: [(0, 0), (1, 1)]
A* Search -->  Path Length: 2, Path: [(0, 0), (1, 1)]

grid 2 : 
Best First Search -->  Path Length: 4, Path: [(0, 0), (0, 1), (1, 2), (2, 2)]
A* Search -->  Path Length: 4, Path: [(0, 0), (0, 1), (1, 2), (2, 2)]

grid 3 : 
Best First Search -->  No Path Found
A* Search -->  No path found


# Difference between BFS and A*

- Best First Search relies only on the heuristic and making decisions based on how close a node appears to be to the goal.

    - faster exploration.
    - does not give optimal solution.
    - less efficient


- A Search*, combines the actual path cost with the heuristic. It balances progress toward the goal.

    - slower than BFS
    - gives optimal solution
    - more efficient